## 使用 POST 方法來爬取公開資訊觀測站上的財務報表
### <p style="line-height:1.6em;" align="justify;">本範例欲爬的目標網站：[公開資訊觀測站](https://mops.twse.com.tw/mops/web/index) （[ 公開資訊觀測站→彙總報表→財務報表→綜合損益表](https://mops.twse.com.tw/mops/#/web/t163sb04)）</p>

In [ ]:
#
# 可以去掉 Python 輸出時，因為軟體版本所引起的警告的警告。
#
import warnings

warnings.filterwarnings('ignore')

###  認識 HTML, CSS, DOM, JavaScript [[1]](https://medium.com/mr-efacani-teatime/%E4%BA%94%E5%88%86%E9%90%98%E8%AA%8D%E8%AD%98css-html-js-2527e3b4ce6e) [[2]](https://5xruby.tw/posts/what-is-html-css-javascript) [[3]](http://blog.ywpu.me/2015/01/21-xmlhtmlxhtmldhtmlhtml-dom.html) [[4*]](https://ithelp.ithome.com.tw/articles/10202326) [[w3school 自學教材]](https://www.w3schools.com/) [[video]](https://www.youtube.com/watch?v=uDgA8yyCnHo&t=1s&ab_channel=%E5%9B%BE%E7%81%B5%E6%98%9F%E7%90%83TuringPlanet)
###  認識 AJAX [[1]](http://www.w3big.com/zh-TW/ajax/ajax-intro.html#gsc.tab=0) [[2]](https://tw.alphacamp.co/blog/ajax-asynchronous-request) [[3]](https://developer.mozilla.org/zh-TW/docs/Web/Guide/AJAX) [[4]](https://linyencheng.github.io/2022/09/22/relationships-between-frontend-and-backend/js-http-client-with-ajax/) [[5]](https://www.youtube.com/watch?v=OkphAk_cWPM) [[6]](https://www.fansysoft.com/web/tw/competency-ajax.tiles) [[7*]](https://blog.tonycube.com/2009/03/ajax.html) [[8*]](https://cs.gmu.edu/~uprapham/classes/swe432/supplement/basic-ajax.html)
<img align="middle" src="pic/ajax.png" width="380"/>

In [ ]:
import requests

#
# 以下 payload 內的 TYPEK, year, season等欄位內容需要做適當設定，說明如下：
# TYPEK(市場別) : 'sii'(上市)，'otc'(上櫃)，'rotc'(興櫃)，'pub'(公開發行)
# year(年度)：以國曆年度為輸入內容
# season(季別)：'01', '02', '03', '04' (第一季至第四季中任選擇一，字串中的'0'不可少)
#
payload = {
    'encodeURIComponent':'1',
    'step':'1',
    'firstin':'1',
    'off':'1',
    'isQuery':'Y',
    'TYPEK':'sii',
    'year':'114',
    'season':'01'
}


# 
# 使用者自訂 http browser request header 中的 user-agent 內容
#
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ,\
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}

#
# 以下為 "公開資訊觀測站→彙總報表→財務報表→採用IFRSs後→綜合損益表" 的網頁網址
#
url_address = 'https://mopsov.twse.com.tw/mops/web/ajax_t163sb04' 


#
# 使用 post 方法
#
res_post = requests.post(url_address, data = payload,  headers = headers)   


#
# 為了能夠順利讓網頁中的中文字正確的呈現出來
#
res_post.encoding = 'utf-8'  

print(res_post.text)

### **用 BeautifulSoup 來剖析並取出所有表格**

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res_post.text, 'html.parser')

#
# 其 html 標籤為 <table class='hasBorder'>...</table>
#
_tables = soup.select("table[class='hasBorder']")  

print('總共有 %d 個表格！\n' % len(_tables))  # 顯示有多少個表格

#print(find_all_tables)

In [ ]:
type(_tables)  # ResultSet class is a subclass of a list and not a Tag class

In [ ]:
_tables

### **用 Pandas 資料科學套件來讀取爬出的表格**

In [ ]:
import pandas as pd

tables = pd.read_html(str(_tables))

print('總共有 %d 個表格！\n' % len(tables)) # 顯示有多少個表格

print(tables)

### **試著依序讀出所抓取的表格，並看一下每一個表格**

In [ ]:
df = tables[0]
df

In [ ]:
tables[1]

In [ ]:
tables[2]

In [ ]:
tables[3]

In [ ]:
tables[4]

In [ ]:
tables[5]

### **將第三個表格內容取出來放入 Pandas DataFrame 內，並看一下表格上有哪一些欄位**** 

In [ ]:
for name in tables[2].columns:
    
    print(name)

### **將索引值 index 起始值從預設的整數值 0 改從整數 1  開始**

In [ ]:
df = pd.DataFrame(tables[2])

df.index += 1

df  # 顯示 df 表格

### Pandas Getting and setting options [[1]](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.set_option.html), [[2]](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html)

In [ ]:
# Note: 更改 DataFrame 顯示的設定
# display.[line_width, max_categories, max_columns, max_colwidth, 
# max_info_columns, max_info_rows, max_rows, max_seq_items, memory_usage, multi_sparse, 
# notebook_repr_html, pprint_nest_depth, precision, show_dimensions]

pd.set_option('display.min_rows', 60)

pd.set_option('display.max_columns', len(tables[2].columns))

#pd.set_option('display.width', 1000)

df  # 顯示 df 表格